## Introduction

The `bfcr` package provides functions to perform nonparametric Bayesian inference for covariate-adjusted covariance operators with functional response data as described in "Bayesian functional covariance regression" by Shamshoian et al. (2021). This package contains a C++ implementation of the Markov-Chain Monte-Carlo sampling algorithm in the Supporting Information document and several convenience functions for extracting posterior summaries. The online GitHub repository contains all code used in the analysis of the resting state EEG data from children with autism spectrum disorder as well as the sleep waveforms collected on patients' polysomnography from SHHS visit 1. However, both of these data sets are not publicly available so we illustrate our methods with a simulated example.

## Installation 

Simply install with GitHub by using the R package `devtools`.

In [ ]:
if (!requireNamespace("devtools", quietly = TRUE)) {
    install.packages("devtools")
  }
# devtools::install_github("jshamsho/bfcr")

## Background info

With the exception of some recent contributions, most functional regression frameworks operate under the assumption of that covariate information does not affect patterns of covariation. The methods in this repository address this disparity providing full nonparametric Bayesian inference on key scientific objects of interest. We assume $N$ response curves are observed with the $i$th response curve having covariate-adjusted mean $\mu(t, x_{i})$, random deviation $r_{i}(t, x_{i})$, and random measurement source error $\epsilon_{i}(t)$. Critically, this framework allows the random component $r_{i}(t, x_{i})$ to systematically vary across scalar covariates $x_{i}$, which induces covariate-adjusted patterns of covariation. The data generating model is 

\begin{gather*}
y_{i}(t) \sim \mu(t, x_{i}) + r_{i}(t, x_{i}) + \epsilon_{i}(t),\,\, \epsilon_{i}(t)\sim N(0, \varphi^{2})\\
r_{i}(t, x_{i}) = \sum_{j=1}^{k}\psi_{j}(t, x_{i})\eta_{ij}, \,\,\eta_{ij}\sim N(0, 1)\\
\mu(t, x_{i}) = \sum_{r=1}^{R}f_{r}(t, x_{i}),\,\,\psi_{j}(t) = \sum_{r=1}^{R}l_{rj}(t, x_{i})
\end{gather*}

Here $f_{r}(t, x_{i})$ and $l_{rj}(t, x_{i})$ are general smooth functions which are in practice estimated via basis expansion. Note that the conditional mean function $\mathbb{E}[y_{i}(t)] = \mu(t, x_{i})$ and the conditional within-subject covariance function is $\text{Cov}[y_{i}(t), y_{i}(t')] = \sum_{j=1}^{k}\psi_{j}(t,x_{i})\psi_{j}(t', x_{i}) + \varphi^{2}\delta_{tt'}$. Estimation on all unknown components is carried out in a Bayesian manner providing full posterior inference. Please see our manuscript for more details on specifying the form of smooth functions, priors, and computation.

## Example

This example will simulate a data set with $N = 100$ response curves and one covariate so that $x_{i}$ is a scalar.

In [10]:
install.packages("mgcv")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

